In [1]:
import configparser
import os
import duckdb

# Set MALLARD_CONFIG environment variable to where the config file is located
os.environ['MALLARD_CONFIG'] = '/data/mallard/config.ini'

# Get config file
config_file = os.getenv('MALLARD_CONFIG')
config = configparser.ConfigParser()
config.read(config_file)

db_file = config['DEFAULT']['db_file']

Here are all the tables in my database. tiingo_fundamentals_amended_distinct is in development, there is a function with description to create it, but it can be ignored for now.

In [2]:
with duckdb.connect(db_file) as con:
    con.sql("SHOW TABLES").show()

┌──────────────────────────────────────┐
│                 name                 │
│               varchar                │
├──────────────────────────────────────┤
│ alembic_version                      │
│ consolidated_metrics                 │
│ daily_metrics                        │
│ inflation                            │
│ tiingo_eod                           │
│ tiingo_fundamental_metrics           │
│ tiingo_fundamentals_amended          │
│ tiingo_fundamentals_amended_distinct │
│ tiingo_fundamentals_daily            │
│ tiingo_fundamentals_last_updated     │
│ tiingo_fundamentals_meta             │
│ tiingo_fundamentals_reported         │
│ tiingo_symbols                       │
├──────────────────────────────────────┤
│               13 rows                │
└──────────────────────────────────────┘



This has symbols taken from supported_tickers.csv, after filters have been applied. Note that Tiingo doesn't include the vendor_symbol_id (permaTicker) in this file.

In [3]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_symbols").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_symbols").show()
    con.sql("SELECT * FROM tiingo_symbols LIMIT 5").show()


┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ symbol         │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ exchange       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ asset_type     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ price_currency │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ start_date     │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ end_date       │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

┌───────────────┐
│ count(symbol) │
│     int64     │
├───────────────┤
│         10111 │
└───────────────┘

┌─────────┬──────────┬────────────┬────────────────┬────────────┬────────────┐
│ symbol

This has metadata for the fundamentals. Even if you don't have the fundamentals addon, we use this to ensure we're getting stocks that are **reporting** fundamentals to the SEC.

In [4]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_meta").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_meta").show()
    con.sql("SELECT * FROM tiingo_fundamentals_meta LIMIT 5").show()

┌────────────────────────┬──────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name       │       column_type        │  null   │   key   │ default │  extra  │
│        varchar         │         varchar          │ varchar │ varchar │ varchar │ varchar │
├────────────────────────┼──────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id       │ VARCHAR                  │ NO      │ PRI     │ NULL    │ NULL    │
│ symbol                 │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ name                   │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ is_active              │ BOOLEAN                  │ YES     │ NULL    │ NULL    │ NULL    │
│ is_adr                 │ BOOLEAN                  │ YES     │ NULL    │ NULL    │ NULL    │
│ sector                 │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ industry               │ VARCHAR                  │ YES   

There are several nuances to fundamentals data, see the docs: https://www.tiingo.com/documentation/fundamentals
You'll want to do a call to the descriptions endpoint to see what all the dataCodes mean.
You'll note I didn't normalize all the columns to make loading a little easier.

Fundamentals amended uses the fiscal period end date and has amendments (corrections) submitted to the SEC.

In [5]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_amended").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_amended").show()
    con.sql("SELECT * FROM tiingo_fundamentals_amended LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ year             │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ quarter          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ accoci           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ acctPay          │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ acctRec          │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ assetsCurrent    │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ assetsNonCurrent │ DOUB

Fundamentals reported uses the filing date, which can be up to 90 days after the fiscal period end date. 
The data is as reported (no corrections) and suitable for backtesting against latest quarter or annual data.
Annual reports have quarter == 0.
Yes, there are ~10% fewer rows than amended and the earliest start date is 1993 vs. 1990. There is no filtering, that's straight from Tiingo. 

In [6]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_reported").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_reported").show()
    con.sql("SELECT * FROM tiingo_fundamentals_reported LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ year             │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ quarter          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ accoci           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ acctPay          │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ acctRec          │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ assetsCurrent    │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ assetsNonCurrent │ DOUB

Fundamentals daily includes metrics generated by Tiingo, not Mallard. Unlike their other fundamental tables, it has a column per metric. 
The docs state they will add new metrics so this may change over time.

In [7]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_daily").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_daily").show()
    con.sql("SELECT * FROM tiingo_fundamentals_daily LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ NO      │ PRI     │ NULL    │ NULL    │
│ market_cap       │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ enterprise_val   │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ pe_ratio         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ pb_ratio         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ trailing_peg_1y  │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

┌───────────────┐
│ coun

Check that the end of day (day bars) table was populated.

In [8]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_eod").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_eod").show()
    con.sql("SELECT * FROM tiingo_eod LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ NO      │ PRI     │ NULL    │ NULL    │
│ close            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ high             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ low              │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ open             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ volume           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ adj_close        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ adj_high         │ DOUB

Daily metrics are calculated by Mallard. It has a metric per column like fundamentals daily and they are created as needed per the config file.
Many metrics are planned, and as an OLAP DB, DuckDB has no problem with very wide tables. 
Note that MACD is a moving average so early values are NULL. 

In [25]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE daily_metrics").show()
    con.sql("SELECT COUNT(symbol) FROM daily_metrics").show()
    con.sql("SELECT * FROM daily_metrics LIMIT 5").show()


┌─────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name       │ column_type │  null   │   key   │ default │  extra  │
│         varchar         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id        │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ symbol                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date                    │ DATE        │ NO      │ PRI     │ NULL    │ NULL    │
│ trading_value           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ avg_daily_trading_value │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ macd                    │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ macd_signal             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ macd_hist               │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ has_min_tradin

Let's verify a symbol exists in all tables.

In [11]:
with duckdb.connect(db_file) as con:
    df = con.execute("""
    SELECT *
    FROM daily_metrics
    WHERE symbol = 'ARR'
    ORDER BY "date" DESC """).df()
df.head()

,vendor_symbol_id,symbol,date,avg_daily_trading_value,has_min_trading_value,macd,macd_signal,macd_hist,grossProfit_ttm_pct_rank,opinc_ttm_pct_rank,...,net_margin_trend_slope_pct_rank,freeCashFlow_trend_slope_pct_rank,currentRatio_trend_slope_pct_rank,debt_trend_slope_pct_rank,debtEquity_trend_slope_pct_rank,roe_trend_slope_pct_rank,bollinger_upper,sma_20,bollinger_lower,rsi
0,US000000004918,ARR,2024-07-05,1.513861e+07,0.0,0.154704,0.157749,-0.003044,29.379175,46.472218,...,0.0,98.305361,0.0,43.865081,51.539840,97.539514,19.730552,19.331644,18.932735,60.112662
1,US000000004918,ARR,2024-07-03,1.574614e+07,0.0,0.139270,0.158510,-0.019240,29.299674,46.449511,...,0.0,98.306189,0.0,43.859935,51.693811,97.573290,19.704481,19.325015,18.945549,56.380908
2,US000000004918,ARR,2024-07-02,1.457572e+07,0.0,0.137908,0.163320,-0.025412,29.278619,46.425664,...,0.0,98.290181,0.0,43.917929,51.717961,97.589969,19.690391,19.306162,18.921933,55.804994
3,US000000004918,ARR,2024-07-01,1.351139e+07,0.0,0.137218,0.169673,-0.032455,29.382114,46.455285,...,0.0,98.276423,0.0,43.772358,51.658537,97.544715,19.677734,19.299180,18.920627,49.629484
4,US000000004918,ARR,2024-06-28,1.568755e+07,0.0,0.164887,0.177786,-0.012900,29.381192,46.467435,...,0.0,98.294624,0.0,43.690109,51.664772,97.579990,19.679622,19.296827,18.914033,55.111829


In [13]:
with duckdb.connect(db_file) as con:
    df = con.execute("""
    SELECT *
    FROM tiingo_fundamentals_reported
    WHERE symbol = 'ARR'
    ORDER BY "date" DESC """).df()
df.head()

,vendor_symbol_id,symbol,date,year,quarter,accoci,acctPay,acctRec,assetsCurrent,assetsNonCurrent,...,sga,shareFactor,sharesBasic,shareswa,shareswaDil,taxAssets,taxExp,taxLiabilities,totalAssets,totalLiabilities
0,US000000004918,ARR,2024-04-25,2024,1,0.0,213056000.0,35948000.0,NaN,NaN,...,1437000.0,1.0,48751806.0,48770000.0,48988000.0,0.0,0.0,0.0,1.220830e+10,1.096118e+10
1,US000000004918,ARR,2024-03-15,2023,4,0.0,178232000.0,0.0,NaN,NaN,...,1262000.0,1.0,48749890.0,48949000.0,48949000.0,0.0,0.0,0.0,1.234440e+10,1.107321e+10
2,US000000004918,ARR,2024-03-15,2023,0,0.0,178232000.0,0.0,NaN,NaN,...,4944000.0,1.0,48749890.0,43054000.0,43054000.0,0.0,0.0,0.0,1.234440e+10,1.107321e+10
3,US000000004918,ARR,2023-10-25,2023,3,0.0,9863000.0,96490000.0,NaN,NaN,...,1262000.0,1.0,48995384.0,46506000.0,46506000.0,0.0,0.0,0.0,1.391420e+10,1.267815e+10
4,US000000004918,ARR,2023-07-26,2023,2,0.0,624881000.0,94825000.0,NaN,NaN,...,1261000.0,1.0,45661847.0,39739800.0,40075800.0,0.0,0.0,0.0,1.253912e+10,1.125613e+10


In [15]:
with duckdb.connect(db_file) as con:
    df = con.execute("""
    SELECT *
    FROM tiingo_fundamentals_meta
    WHERE symbol = 'ARR'""").df()
df.head()

,vendor_symbol_id,symbol,name,is_active,is_adr,sector,industry,sic_code,sic_sector,sic_industry,reporting_currency,location,company_website,sec_filing_website,statement_last_updated,daily_last_updated,vendor_entity_id
0,US000000004918,ARR,ARMOUR Residential REIT Inc,True,False,Real Estate,REIT - Mortgage,6798,Finance Insurance And Real Estate,Real Estate Investment Trusts,USD,"Florida, USA",http://www.armourreit.com,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2024-04-26 01:09:33.998972-07:00,2024-07-06 01:12:58.142926-07:00,193597


In [9]:
with duckdb.connect(db_file) as con:
    df = con.execute("""
    SELECT *
    FROM tiingo_fundamental_metrics
    WHERE symbol = 'ARR'""").df()
df.head()

,vendor_symbol_id,symbol,date,year,quarter,grossProfit_ttm,opinc_ttm,netinc_ttm,revenue_ttm,gross_margin_ttm,...,roe_trend_slope,grossProfit_trend_slope,opinc_trend_slope,netinc_trend_slope,revenue_trend_slope,gross_margin_trend_slope,operating_margin_trend_slope,net_margin_trend_slope,freeCashFlow_trend_slope,ebitda_trend_slope
0,US000000004918,ARR,2024-04-25,2024,1,-198000.0,-22048000.0,-10066000.0,31949000.0,-0.006197,...,2.370253,0.281624,0.323986,0.191165,0.224229,NaN,NaN,NaN,4.257488,0.191165
1,US000000004918,ARR,2024-03-15,2023,4,-214392000.0,-235488000.0,-214520000.0,-159399000.0,NaN,...,0.132709,-0.135966,-0.142449,-0.132653,-0.093441,NaN,NaN,NaN,0.582445,-0.132653
2,US000000004918,ARR,2023-10-25,2023,3,-116250000.0,-128155000.0,-116173000.0,-85992000.0,NaN,...,0.072796,0.008418,0.007240,0.011042,0.026802,NaN,NaN,NaN,0.552892,0.011042
3,US000000004918,ARR,2023-07-26,2023,2,-81398000.0,-93302000.0,-81320000.0,-52625000.0,NaN,...,0.099623,0.059691,0.055847,0.061966,0.081459,NaN,NaN,NaN,0.626491,0.061966
4,US000000004918,ARR,2023-04-26,2023,1,-183637000.0,-194855000.0,-182873000.0,-156259000.0,NaN,...,-0.018885,-0.090720,-0.085781,-0.086413,-0.088298,NaN,NaN,NaN,0.648444,-0.086413


In [16]:
with duckdb.connect(db_file) as con:
    df = con.execute("""
    SELECT *
    FROM tiingo_fundamentals_daily
    WHERE symbol = 'ARR'
    ORDER BY "date" DESC """).df()
df.head()

,vendor_symbol_id,symbol,date,market_cap,enterprise_val,pe_ratio,pb_ratio,trailing_peg_1y
0,US000000004918,ARR,2024-07-05,959397835.2,9.353944e+09,-28.192707,0.769295,0.225068
1,US000000004918,ARR,2024-07-03,949647857.2,9.344194e+09,-27.906196,0.761477,0.222781
2,US000000004918,ARR,2024-07-02,948185360.5,9.342731e+09,-27.863220,0.760304,0.222437
3,US000000004918,ARR,2024-07-01,933560393.5,9.328106e+09,-27.433453,0.748577,0.219007
4,US000000004918,ARR,2024-06-28,944772868.2,9.339319e+09,-27.762941,0.757568,0.221637


In [3]:
from mallard.metrics.slope_util import get_raw_income

with duckdb.connect(db_file) as con:
    fundamental_metrics_table = config['tiingo']['fundamental_metrics_table']
    
    rows_processed = 0
    balance_sheet_metrics = ['currentRatio', 'debt', 'debtEquity', 'piotroskiFScore', 'roe']
    column_names = con.sql(f"SELECT * FROM {fundamental_metrics_table} LIMIT 1").columns
    
    income_statement_metrics = [col for col in column_names if col.endswith("_ttm")]
    combined_metrics = balance_sheet_metrics + income_statement_metrics
    
    df = get_raw_income(con, income_statement_metrics)
df.head()

,vendor_symbol_id,year,quarter,date,grossProfit_ttm,grossProfit_y1,grossProfit_y2,grossProfit_y3,opinc_ttm,opinc_y1,...,net_margin_y2,net_margin_y3,freeCashFlow_ttm,freeCashFlow_y1,freeCashFlow_y2,freeCashFlow_y3,ebitda_ttm,ebitda_y1,ebitda_y2,ebitda_y3
0,US000000000038,1995,1,1995-02-09,2.565000e+09,2.343000e+09,2.728000e+09,3.095205e+09,742000000.0,522000000.0,...,0.010906,0.074842,662000000.0,577000000.0,-864000000.0,726120000.0,898000000.0,668000000.0,306000000.0,1.072624e+09
1,US000000000038,1995,2,1995-05-15,2.761000e+09,2.343000e+09,2.728000e+09,3.095205e+09,873000000.0,522000000.0,...,0.010906,0.074842,731000000.0,577000000.0,-864000000.0,726120000.0,977000000.0,668000000.0,306000000.0,1.072624e+09
2,US000000000038,1995,3,1995-08-11,2.915000e+09,2.343000e+09,2.728000e+09,3.095205e+09,802000000.0,522000000.0,...,0.010906,0.074842,528000000.0,577000000.0,-864000000.0,726120000.0,914000000.0,668000000.0,306000000.0,1.072624e+09
3,US000000000038,1995,4,1995-12-19,2.858000e+09,2.343000e+09,2.728000e+09,3.095205e+09,684000000.0,522000000.0,...,0.010906,0.074842,-399000000.0,577000000.0,-864000000.0,726120000.0,801000000.0,668000000.0,306000000.0,1.072624e+09
4,US000000000038,1996,1,1996-02-12,2.519000e+09,2.858000e+09,2.343000e+09,2.728000e+09,281000000.0,684000000.0,...,0.033736,0.010906,-674000000.0,-399000000.0,577000000.0,-864000000.0,397000000.0,801000000.0,668000000.0,3.060000e+08


In [4]:

def get_sid_by_symbol(symbol, is_active=True):
    with duckdb.connect(db_file) as con:
        query = f"""
        SELECT vendor_symbol_id FROM tiingo_fundamentals_meta
        WHERE symbol = '{symbol}' AND is_active = {is_active};
"""
        return con.execute(query).fetchall()[0][0]
    

In [5]:
print(get_sid_by_symbol('ROST'))

US000000000400


In [6]:
# print(df[df['vendor_symbol_id'] == 'US000000009517'])

# From df, get most recent row by date where vendor_symbol_id = US000000009517
def get_most_recent_row(df, sid):
    return df[df['vendor_symbol_id'] == sid].sort_values('date', ascending=False).iloc[0]

mr_df = get_most_recent_row(df, get_sid_by_symbol('ROST'))

In [7]:
# From urg_df, get date and all columns that contain 'net'
mr_df.filter(like='netinc')


netinc_ttm    1874520000.0
netinc_y1     1512041000.0
netinc_y2     1722589000.0
netinc_y3       85382000.0
Name: 26286, dtype: object

In [8]:
import numpy as np

y = np.array([mr_df['netinc_y3'], mr_df['netinc_y2'], mr_df['netinc_y1'], mr_df['netinc_ttm']])
x = np.array([1, 2, 3, 4])  # Time points
first_non_zero = next((val for val in y if val != 0 and not np.isnan(val)), None)
y_normalized = y / abs(first_non_zero)
slope, _ = np.polyfit(x, y_normalized, 1)
slope

6.039757794382892

In [33]:
y = np.array([mr_df['netinc_y3'], mr_df['netinc_y2'], mr_df['netinc_y1'], mr_df['netinc_ttm']])
y

array([-2.9000e+04, -1.4109e+07,  4.1325e+07, -6.1510e+06])